In [124]:
# compute possible pin combinations
# input: observed pin, keyboard, width (how far to permutate a specific character in the observed pin)
# output: possible pin combinations


import pandas as pd


# keyboard characters must be all unique
# all columns and rows must be integers
def to_keyboard(keyb_list, kind):
    keyboard = []
    for keyb_line in keyb_list:
        keyboard.append([k for k in keyb_line])
    # all columns must be strings not int    
    keyboard_df = pd.DataFrame(keyboard, dtype='object')
    # be careful of the order you have passed the keyb_list in. 
    # The order of the list might not be the actual order in the real keyboard
    # if exceptions, specify here
    if kind == 'nokia':
        keyboard_df.iloc[3][0] = None
        keyboard_df.iloc[3][1] = '0'
    return keyboard_df


    
keyboard_nokia = to_keyboard([
    '123', 
    '456',
    '789',
    '0'
], 'nokia')



keyboard_qwerty = to_keyboard([
    '1234567890',
    'qwertyuiop',
    'asdfghjkl',
    'zxcvbnm'    
], 'qwerty')


In [125]:


# ouputs all adjacent characters around each character
def map_keyboard(keyboard):
    
    result = []
    tot_cols = len(keyboard.columns)
    tot_rows = len(keyboard)
    
    for x, this_row in keyboard.iterrows():        
        for y in keyboard:
            c = this_row[y]
            if c == None:
                continue
            
            if y-1 < 0:
                l = None
                r = keyboard[y+1][x]
            elif y+1 >= tot_cols:
                l = keyboard[y-1][x]
                r = None
            elif y-1 >= 0 and y+1 < tot_cols:
                l = keyboard[y-1][x]
                r = keyboard[y+1][x]

            if x-1 < 0:
                u = None
                d = keyboard[y][x+1]
            elif x+1 >= tot_rows:
                u = keyboard[y][x-1]
                d = None
            elif x-1 >= 0 and x+1 < tot_rows:
                u = keyboard[y][x-1]
                d = keyboard[y][x+1]

            result.append([c, x, y, l, r, u, d])
    
    return pd.DataFrame(result, columns=['c', 'x', 'y', 'l', 'r', 'u', 'd'])


keyboard_map = map_keyboard(keyboard_qwerty)



def get_info(c):
    return lambda what: keyboard_map[keyboard_map['c'] == c][what].values.tolist()[0]




keyboard_map

,c,x,y,l,r,u,d
0,1,0,0,None,2,None,q
1,2,0,1,1,3,None,w
2,3,0,2,2,4,None,e
3,4,0,3,3,5,None,r
4,5,0,4,4,6,None,t
5,6,0,5,5,7,None,y
6,7,0,6,6,8,None,u
7,8,0,7,7,9,None,i
8,9,0,8,8,0,None,o
9,0,0,9,9,None,None,p


In [126]:
# output all values to be permuted
def get_values_perm(chars, keyboard):
    to_permute = []
    chars_list = [a for a in chars]
    for char in chars_list:
        info = get_info(char)
        l = info('l')
        r = info('r')
        u = info('u')
        d = info('d')
        to_permute.append([char, l, r, u, d])
    return to_permute
    
    
# permute one level at a time for each level
def permute_one(chars, keyboard):
    to_permute = get_values_perm(chars, keyboard)
    len_to_permute = len(to_permute)
    perms = []
    
    print(to_permute)
    
    for i_row, this_row in enumerate(to_permute):
        if i_row+1 == len_to_permute:
            break
        for p in this_row:
            if p == None:
                continue
            perms.append([])
            next_row = to_permute[i_row+1]
            for next_p in next_row:
                if next_p == None:
                    continue
                perms[-1].append(p + next_p)
            
    return perms
             


permute_one('dog', keyboard_qwerty)


[['d', 's', 'f', 'e', 'c'], ['o', 'i', 'p', '9', 'l'], ['g', 'f', 'h', 't', 'b']]


[['do', 'di', 'dp', 'd9', 'dl'],
 ['so', 'si', 'sp', 's9', 'sl'],
 ['fo', 'fi', 'fp', 'f9', 'fl'],
 ['eo', 'ei', 'ep', 'e9', 'el'],
 ['co', 'ci', 'cp', 'c9', 'cl'],
 ['og', 'of', 'oh', 'ot', 'ob'],
 ['ig', 'if', 'ih', 'it', 'ib'],
 ['pg', 'pf', 'ph', 'pt', 'pb'],
 ['9g', '9f', '9h', '9t', '9b'],
 ['lg', 'lf', 'lh', 'lt', 'lb']]

In [127]:
# second version

def permute_all(chars, keyboard):
    to_permute = get_values_perm(chars, keyboard)
    size_to_permute = len(to_permute)
    perms = [[]]
    
    def perm(i_row, row, prev_p):
         if i_row+1 == size_to_permute:           
            for p in row:
                if p == None: 
                    continue
                perms[-1].append(prev_p+p)
            perms.append([])
            return   
         for p in row:
            if p == None: 
                continue
            perm(i_row+1, to_permute[i_row+1], prev_p+p)
            
    perm(0, to_permute[0], '')
    perms.pop()
    print(perms)
    return perms
             

    
permutations_all = permute_all('dog', keyboard_qwerty)
permutations_all


[['dog', 'dof', 'doh', 'dot', 'dob'], ['dig', 'dif', 'dih', 'dit', 'dib'], ['dpg', 'dpf', 'dph', 'dpt', 'dpb'], ['d9g', 'd9f', 'd9h', 'd9t', 'd9b'], ['dlg', 'dlf', 'dlh', 'dlt', 'dlb'], ['sog', 'sof', 'soh', 'sot', 'sob'], ['sig', 'sif', 'sih', 'sit', 'sib'], ['spg', 'spf', 'sph', 'spt', 'spb'], ['s9g', 's9f', 's9h', 's9t', 's9b'], ['slg', 'slf', 'slh', 'slt', 'slb'], ['fog', 'fof', 'foh', 'fot', 'fob'], ['fig', 'fif', 'fih', 'fit', 'fib'], ['fpg', 'fpf', 'fph', 'fpt', 'fpb'], ['f9g', 'f9f', 'f9h', 'f9t', 'f9b'], ['flg', 'flf', 'flh', 'flt', 'flb'], ['eog', 'eof', 'eoh', 'eot', 'eob'], ['eig', 'eif', 'eih', 'eit', 'eib'], ['epg', 'epf', 'eph', 'ept', 'epb'], ['e9g', 'e9f', 'e9h', 'e9t', 'e9b'], ['elg', 'elf', 'elh', 'elt', 'elb'], ['cog', 'cof', 'coh', 'cot', 'cob'], ['cig', 'cif', 'cih', 'cit', 'cib'], ['cpg', 'cpf', 'cph', 'cpt', 'cpb'], ['c9g', 'c9f', 'c9h', 'c9t', 'c9b'], ['clg', 'clf', 'clh', 'clt', 'clb']]


[['dog', 'dof', 'doh', 'dot', 'dob'],
 ['dig', 'dif', 'dih', 'dit', 'dib'],
 ['dpg', 'dpf', 'dph', 'dpt', 'dpb'],
 ['d9g', 'd9f', 'd9h', 'd9t', 'd9b'],
 ['dlg', 'dlf', 'dlh', 'dlt', 'dlb'],
 ['sog', 'sof', 'soh', 'sot', 'sob'],
 ['sig', 'sif', 'sih', 'sit', 'sib'],
 ['spg', 'spf', 'sph', 'spt', 'spb'],
 ['s9g', 's9f', 's9h', 's9t', 's9b'],
 ['slg', 'slf', 'slh', 'slt', 'slb'],
 ['fog', 'fof', 'foh', 'fot', 'fob'],
 ['fig', 'fif', 'fih', 'fit', 'fib'],
 ['fpg', 'fpf', 'fph', 'fpt', 'fpb'],
 ['f9g', 'f9f', 'f9h', 'f9t', 'f9b'],
 ['flg', 'flf', 'flh', 'flt', 'flb'],
 ['eog', 'eof', 'eoh', 'eot', 'eob'],
 ['eig', 'eif', 'eih', 'eit', 'eib'],
 ['epg', 'epf', 'eph', 'ept', 'epb'],
 ['e9g', 'e9f', 'e9h', 'e9t', 'e9b'],
 ['elg', 'elf', 'elh', 'elt', 'elb'],
 ['cog', 'cof', 'coh', 'cot', 'cob'],
 ['cig', 'cif', 'cih', 'cit', 'cib'],
 ['cpg', 'cpf', 'cph', 'cpt', 'cpb'],
 ['c9g', 'c9f', 'c9h', 'c9t', 'c9b'],
 ['clg', 'clf', 'clh', 'clt', 'clb']]